# 📊 Healthcare ABAC Demo - Step 3: Setup Governed Tags

## 📋 Overview
This notebook creates the tag policies defining the necessary governed tags and their allowed values

### What This Notebook Does:
1. **Creates Tag policy**: Uses REST API to create tag policies defining governed tags and their allowed values
2. **Demonstrates Usage**: Shows how to apply tags to relevant columns in the healthcare datasets

## 🎓 How to Use This Notebook
1. **Ensure Steps 1-2 Complete**: All functions, tables, and data must exist
2. **Run All Cells**: Execute sequentially to see all test results
3. **Verify Expectations**: Check that tags have been applied through the catalog explorer

## ⚙️ Prerequisites
- ✅ **Step 1 completed**: All functions created
- ✅ **Step 2 completed**: Core tables with data
- ✅ APPLY TAG permission on all tables

## 📊 Expected Results
After running this notebook:
- ✅ Tag policies are created
- ✅ Governed tags assigned
- ✅ Prepared for creating ABAC policies


In [0]:
pip install pyyaml

## ⚙️ Configuration

Using the same catalog and schema from previous steps:
- **Catalog**: `your_catalog_name`
- **Schema**: `healthcare`


In [0]:
# 📋 Load Configuration from config.yaml
import yaml
from pathlib import Path
import requests
import os
from databricks.sdk import WorkspaceClient

config_file = Path('config.yaml')
if config_file.exists():
    with open(config_file) as f:
        config = yaml.safe_load(f)
    CATALOG = config['catalog']
    SCHEMA = config['schema']
    print(f'✅ Configuration loaded from config.yaml')
    print(f'   📊 Catalog: {CATALOG}')
    print(f'   📁 Schema: {SCHEMA}')
else:
    # Fallback defaults
    CATALOG = 'your_catalog_name'
    SCHEMA = 'insurance'
    print(f'⚠️  config.yaml not found - using defaults')
    print(f'   📊 Catalog: {CATALOG}')
    print(f'   📁 Schema: {SCHEMA}')

# Set catalog and schema to use for the cells below
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {SCHEMA}")
spark.sql(f"USE SCHEMA {SCHEMA}")

client = WorkspaceClient()
workspace_url = client.config.host

In [0]:
%sql
SELECT '🎯 Target: ' || current_catalog() || '.' || current_schema() AS status;

###Define Governed Tags + Allowed Values
https://docs.databricks.com/api/workspace/tagpolicies/createtagpolicy

In [0]:
def get_token():
    ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
    return getattr(ctx, "apiToken")().get()


def create_tag_policy(payload):
    data = requests.post(
        f"{workspace_url}/api/2.1/tag-policies",
        headers={"Authorization": f"Bearer {get_token()}"},
        json=payload,
    )

    return data

In [0]:
pii_payload = {
    "description": "Healthcare PII data types",
    "tag_key": "pii_type_healthcare",
    "values": [
        {"name": "patient_name"},
        {"name": "patient_id"},
        {"name": "ssn"},
        {"name": "dob"},
        {"name": "email"},
        {"name": "phone"},
        {"name": "address"},
        {"name": "medical_record_number"},
        {"name": "diagnosis"},
        {"name": "treatment"}
    ]
}
print(create_tag_policy(pii_payload).json())

phi_payload = {
    "description": "Protected Health Information level",
    "tag_key": "phi_level_healthcare",
    "values": [
        {"name": "High"},
        {"name": "Medium"},
        {"name": "Low"},
        {"name": "Public"}
    ]
}
print(create_tag_policy(phi_payload).json())

hipaa_payload = {
    "description": "HIPAA compliance requirement",
    "tag_key": "hipaa_compliance_healthcare",
    "values": [
        {"name": "Required"},
        {"name": "Not_Required"}
    ]
}
print(create_tag_policy(hipaa_payload).json())

sensitivity_payload = {
    "description": "Data sensitivity classification",
    "tag_key": "data_sensitivity_healthcare",
    "values": [
        {"name": "Highly_Sensitive"},
        {"name": "Sensitive"},
        {"name": "Internal"},
        {"name": "Public"}
    ]
}
print(create_tag_policy(sensitivity_payload).json())

shift_payload = {
    "description": "Time-based access control for healthcare operations (Standard_Business=8AM-6PM)",
    "tag_key": "shift_hours_healthcare",
    "values": [
        {"name": "Standard_Business"},
        {"name": "Extended_Hours"},
        {"name": "Night_Shift"},
        {"name": "Emergency_24x7"}
    ]
}
print(create_tag_policy(shift_payload).json())

In [0]:
%sql
ALTER TABLE lab_results SET TAGS ('hipaa_compliance_healthcare' = 'Required');
ALTER TABLE lab_results ALTER COLUMN ResultValue SET TAGS ('phi_level_healthcare' = 'High', 'shift_hours_healthcare' = 'Standard_Business');
ALTER TABLE lab_results ALTER COLUMN TestName SET TAGS ('phi_level_healthcare' = 'Medium', 'shift_hours_healthcare' = 'Standard_Business');
ALTER TABLE lab_results ALTER COLUMN PatientID SET TAGS ('pii_type_healthcare' = 'patient_id', 'phi_level_healthcare' = 'High');

In [0]:
%sql
ALTER TABLE billing SET TAGS ('hipaa_compliance_healthcare' = 'Not_Required');
ALTER TABLE billing ALTER COLUMN ChargeAmount SET TAGS ('data_sensitivity_healthcare' = 'Sensitive');

In [0]:
%sql
ALTER TABLE patients SET TAGS ('hipaa_compliance_healthcare' = 'Required');
ALTER TABLE patients ALTER COLUMN FirstName SET TAGS ('pii_type_healthcare' = 'patient_name');
ALTER TABLE patients ALTER COLUMN LastName SET TAGS ('pii_type_healthcare' = 'patient_name');
ALTER TABLE patients ALTER COLUMN PhoneNumber SET TAGS ('pii_type_healthcare' = 'phone','data_sensitivity_healthcare' = 'Sensitive');
ALTER TABLE patients ALTER COLUMN Email SET TAGS ('pii_type_healthcare' = 'email','data_sensitivity_healthcare' = 'Internal');
ALTER TABLE patients ALTER COLUMN DateOfBirth SET TAGS ('pii_type_healthcare' = 'dob', 'phi_level_healthcare' = 'High');
ALTER TABLE patients ALTER COLUMN PatientID SET TAGS ('pii_type_healthcare' = 'patient_id', 'phi_level_healthcare' = 'High');

In [0]:
%sql
ALTER TABLE insurance SET TAGS ('hipaa_compliance_healthcare' = 'Not_Required');
ALTER TABLE insurance ALTER COLUMN PolicyNumber SET TAGS ('data_sensitivity_healthcare' = 'Sensitive');
ALTER TABLE insurance ALTER COLUMN GroupNumber SET TAGS ('data_sensitivity_healthcare' = 'Sensitive');
ALTER TABLE insurance ALTER COLUMN PatientID SET TAGS ('pii_type_healthcare' = 'patient_id', 'phi_level_healthcare' = 'High');

In [0]:
%sql
SELECT '✅ Tags applied successfully to healthcare tables!' AS status;

## ✅ Success!

Tag policies have been created successfully and governed tags have been assigned!

### What You Just Created:
- ✅ Governed tags for capturing data sensitivity 
- ✅ Tag assignment to tables

### 🎯 Next Step:

Continue to **`4_Test_ABAC_Policies.ipynb`** to define ABAC policies using governed tags and test them on each dataset in the healthcare schema

---